In [102]:
#Generating Data for Training and Test
import numpy as np

N = 10000
a_A = np.array([i/1. for i in range(N)])+10*np.random.rand(N)
a_B = np.array([i/1. for i in range(N)])+10*np.random.rand(N)
temp1, temp2 = [], []

for i in range(N):
    temp1.append([a_A[i],a_B[i],a_A[i]+a_B[i]]) # data for add : [A,B,A+B]
    temp2.append([a_A[i],a_B[i],a_A[i]*a_B[i]]) # data for multiply : [A,B,A*B]

data_madd = np.array(temp1)
data_mmul = np.array(temp2)

np.save('datatrain_madd.npy', data_madd)
np.save('datatrain_mmul.npy', data_mmul)
    
arr_A = 2*np.random.randn(N)+1
arr_B = 2*np.random.randn(N)+1

A = 10**(arr_A) # A data
B = 10**(arr_B) # B data

for i in range(N):
    if (arr_A[i] <= 4 and arr_A[i] >= -4) and (arr_B[i] <= 4 and arr_B[i] >= -4):
        temp1.append([A[i],B[i],A[i]+B[i]]) # data for add : [A,B,A+B]
        temp2.append([A[i],B[i],A[i]*B[i]]) # data for multiply : [A,B,A*B]

data_madd = np.array(temp1)
data_mmul = np.array(temp2)

np.save('datatest_madd.npy', data_madd)
np.save('datatest_mmul.npy', data_mmul)



#data3 = np.load('my_data.npy') # 데이터 로드. @파일명

In [108]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from itertools import cycle, islice
from time import sleep
import sys
import myNN

%matplotlib inline
%reload_ext autoreload
%autoreload 2

data_madd = np.load('datatrain_madd.npy') #load data for add_train
data_mmul = np.load('datatrain_mmul.npy') #load data for mul_train

testdata_madd = np.load('datatest_madd.npy') #load data for add_test
testdata_mmul = np.load('datatest_mmul.npy') #load data for mul_test

#Set model structure and learning rate
model_str = np.array(['2:identity|'+'2:identity|'+'1:identity', \
             '2:identity|'+'2:log|'+'2:identity|'+'2:exp|'+'1:identity'])
lr = np.array([0.001, 0.001])
name_tag = np.array(['add_lr'+str(lr[0]), 'mul_lr'+str(lr[1])])


#Call myNN class
madd = myNN.madd(model_structure=model_str[0], \
                     model_nametag=name_tag[0], \
                     learning_rate=lr[0], \
                     encoding='float')

# 훈련 준비
data_type=None # None
encoding='float' #'integer' # 'float'

# 훈련 & 테스트 데이터 준비
# n_data_tot=len(training_data_list)
n_data_max=10000  # 훈련에 사용할 데이터 갯수 (max = 10000)
n_data_test=2000  # 테스트에 사용할 테이터 갯수 (10000 - n_data_max)

## 훈련 데이터
training_data_list = data_madd[:n_data_max]

## 테스트 데이터
test_data_list = testdata_madd[-n_data_test:]

# 최대학습주기 설정
epochs = 100 #10000
dn_data = int(n_data_max/20)

# 설정된 최대학습주기동안 훈련

for e in range(epochs):

    id_data = 0
    
    for data in training_data_list:
        
        # 프로세스 게이지
        id_data += 1
        if (id_data%dn_data==0):
            sys.stdout.write('\r')
            sys.stdout.write(' [%-20s] %d%%' % ( '='*(id_data//dn_data), 5*(id_data//dn_data)))
            sys.stdout.flush()
            sleep(0.25)
        
        # 입력/지도 데이터 가공 
        input_list = data[:2] #np.asfarray(all_values[1:])
        target_origin = data[2]
        target_list = np.zeros(1)
        target_list[0] = data[2]
        
        madd.train(input_list, target_list)
        
        pass

    #열번마다 정확도와 평균 에러를 표시해주면서, nnfactory안에 error를 기록해놓습니다.
    if (e%10==0):
        print (' --------------------------------------')
        print(' * epoch = {}'.format(e+1))
        print(' > 훈련 샘플에 대한 성능 (정확도 & 평균에러) ')
        #mynn.check_accuracy_error에서 현재 정확도와 평균에러를 return하도록 설정해주어 이를 training_err로 받은 모습입니다.
        madd.check_accuracy_error(training_data_list, 0, n_data_max-1, data_type=None)
        print('')
        print(' > 테스트 샘플에 대한 성능 (정확도 & 평균에러) ')
        #마찬가지로, mynn.check_accuracy_error에서 현재 정확도와 평균에러를 return하도록 설정해주어 이를 test_err로 받은 모습입니다.
        madd.check_accuracy_error(test_data_list, 0, n_data_test-1, data_type=None)
    
    pass

              


 * 다음과 같은 구조의 다층퍼셉트론 연결이 초기화 되었습니다 *

 > 모델이름 = add_lr0.001
 > 총 층수 (입력 + 은닉(s) + 출력) =  3
 > 각 층에서의 노드수 =  [2, 2, 1]
 > 각 층에서의 활성화 함수 =  ['identity', 'identity', 'identity']
 > 학습률(Learning Rate) =  0.001
 > 지도라벨 인코딩 방식 =  float
 [====================] 100% --------------------------------------
 * epoch = 1
 > 훈련 샘플에 대한 성능 (정확도 & 평균에러) 
 
 * 현재 평균에러 =  inf

 > 테스트 샘플에 대한 성능 (정확도 & 평균에러) 
 
 * 현재 평균에러 =  inf
 [=========           ] 45%

KeyboardInterrupt: 

In [111]:

mmul = myNN.mmul(model_structure=model_str[1], \
                     model_nametag=name_tag[1], \
                     learning_rate=lr[1], \
                     encoding='float',optimizer = 'momentum')

# 훈련 준비
data_type=None # None
encoding='float' #'integer' # 'float'

# 훈련 & 테스트 데이터 준비
# n_data_tot=len(training_data_list)
n_data_max=10000  # 훈련에 사용할 데이터 갯수 (max = 10000)
n_data_test=2000  # 테스트에 사용할 테이터 갯수 (10000 - n_data_max)

## 훈련 데이터
training_data_list = data_madd[:n_data_max]

## 테스트 데이터
test_data_list = testdata_madd[-n_data_test:]

# 최대학습주기 설정
epochs = 100 #10000
dn_data = int(n_data_max/20)

# 설정된 최대학습주기동안 훈련

for e in range(epochs):

    id_data = 0
    
    for data in training_data_list:
        
        # 프로세스 게이지
        id_data += 1
        if (id_data%dn_data==0):
            sys.stdout.write('\r')
            sys.stdout.write(' [%-20s] %d%%' % ( '='*(id_data//dn_data), 5*(id_data//dn_data)))
            sys.stdout.flush()
            sleep(0.25)
        
        # 입력/지도 데이터 가공 
        input_list = data[:2] #np.asfarray(all_values[1:])
        target_origin = data[2]
        target_list = np.zeros(1)
        target_list[0] = data[2]
        
        mmul.train(input_list, target_list)
        
        pass

    #열번마다 정확도와 평균 에러를 표시해주면서, nnfactory안에 error를 기록해놓습니다.
    if (e%10==0):
        print (' --------------------------------------')
        print(' * epoch = {}'.format(e+1))
        print(' > 훈련 샘플에 대한 성능 (정확도 & 평균에러) ')
        #mynn.check_accuracy_error에서 현재 정확도와 평균에러를 return하도록 설정해주어 이를 training_err로 받은 모습입니다.
        mmul.check_accuracy_error(training_data_list, 0, n_data_max-1, data_type=None)
        print('')
        print(' > 테스트 샘플에 대한 성능 (정확도 & 평균에러) ')
        #마찬가지로, mynn.check_accuracy_error에서 현재 정확도와 평균에러를 return하도록 설정해주어 이를 test_err로 받은 모습입니다.
        mmul.check_accuracy_error(test_data_list, 0, n_data_test-1, data_type=None)
    
    pass

              


 * 다음과 같은 구조의 다층퍼셉트론 연결이 초기화 되었습니다 *

 > 모델이름 = mul_lr0.001
 > 총 층수 (입력 + 은닉(s) + 출력) =  5
 > 각 층에서의 노드수 =  [2, 2, 2, 2, 1]
 > 각 층에서의 활성화 함수 =  ['identity', 'log', 'identity', 'exp', 'identity']
 > 학습률(Learning Rate) =  0.001
 > 지도라벨 인코딩 방식 =  float
4
3


ValueError: shapes (2,) and (1,1) not aligned: 2 (dim 0) != 1 (dim 0)

In [ ]:
#Save madd & mmul model
madd.save_model(fname='madd_2014-19402.npz', nametag='madd_4relu')
mmul.save_model(fname='mmul_2014-19402.npz', nametag='mmul_4relu')